In [1]:
import sys

sys.path.append('../GTS/GTS')

In [3]:
import torch
import numpy as np
import pandas as pd
import os
import time
from glob import glob
from itertools import cycle

In [4]:
dirs = '/Users/juhyeonkim/Documents/graphNN/M5_Dataset'

In [66]:
INPUT_DIR = dirs

calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
calendar.name = 'calendar'

prices = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')
prices.name = 'prices'

sales = pd.read_csv(f'{INPUT_DIR}/sales_train_evaluation.csv')
sales.name = 'sales'

In [67]:
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

<ipython-input-67-02f8d3558dac>:21: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif t == np.object:


In [68]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [69]:
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


In [70]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [71]:
len(sales)

30490

In [72]:
hobby_1 = sales[sales.dept_id == 'HOBBIES_1']

In [73]:
hobby_1

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27852,HOBBIES_1_420_WI_3_evaluation,HOBBIES_1_420,HOBBIES_1,HOBBIES,WI_3,WI,1,1,0,3,...,0,1,1,1,1,1,0,2,0,0
27853,HOBBIES_1_421_WI_3_evaluation,HOBBIES_1_421,HOBBIES_1,HOBBIES,WI_3,WI,0,0,0,0,...,0,1,1,1,1,0,0,0,0,0
27854,HOBBIES_1_422_WI_3_evaluation,HOBBIES_1_422,HOBBIES_1,HOBBIES,WI_3,WI,1,2,0,0,...,0,0,0,0,0,0,0,1,0,0
27855,HOBBIES_1_423_WI_3_evaluation,HOBBIES_1_423,HOBBIES_1,HOBBIES,WI_3,WI,0,0,0,0,...,0,0,2,1,0,0,1,1,0,0


In [74]:
sales.groupby(sales['dept_id']).mean()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
dept_id,,,,,,,,,,,,,,,,,,,,,
FOODS_1,1.084722,1.025926,0.767130,0.698148,0.559722,0.878241,0.881019,1.034722,0.891204,0.734259,...,1.912037,1.970833,1.956019,1.511111,1.443981,1.422685,1.505093,1.729167,1.942130,1.866204
FOODS_2,1.028643,1.057538,0.797487,0.906030,0.720854,1.099246,0.841457,1.101508,1.073618,0.925628,...,1.467839,1.900251,2.278643,1.564322,1.334422,1.167839,1.143467,1.209799,1.492462,1.641960
FOODS_3,2.034143,1.984569,1.499757,1.672418,1.278858,1.922357,1.851519,2.567679,2.267801,1.656258,...,2.446537,3.008019,3.420778,2.389793,2.152855,2.073998,2.057716,2.379830,2.851033,3.086391
HOBBIES_1,0.867788,0.762500,0.600240,0.608413,0.412019,0.753125,0.686298,0.920913,0.658894,0.604327,...,0.822596,1.061058,1.073077,0.747356,0.755288,0.791106,0.727644,0.904087,1.161538,1.134615
HOBBIES_2,0.103356,0.124161,0.124161,0.092617,0.067114,0.058389,0.059732,0.104027,0.106040,0.067785,...,0.301342,0.321477,0.370470,0.281208,0.326846,0.301342,0.300671,0.256376,0.336242,0.375839
HOUSEHOLD_1,0.771617,0.725188,0.531391,0.513534,0.338722,0.500752,0.594173,0.814662,0.656767,0.537030,...,1.491729,1.887594,1.987218,1.430263,1.315602,1.248308,1.270489,1.498872,1.817293,1.910714
HOUSEHOLD_2,0.307573,0.344854,0.213592,0.220000,0.180000,0.239612,0.274757,0.361359,0.287573,0.232816,...,0.419223,0.543107,0.560388,0.401359,0.388544,0.373786,0.386796,0.446408,0.566602,0.568155


In [75]:
sales.groupby(sales['store_id']).mean()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
store_id,,,,,,,,,,,,,,,,,,,,,
CA_1,1.422434,1.362742,0.923582,1.000656,0.862578,1.074451,1.131519,1.783208,1.423418,1.035421,...,1.623811,2.048213,2.199738,1.498196,1.295179,1.310266,1.356510,1.453919,1.890456,2.062643
CA_2,1.145949,0.999016,0.695638,0.762217,0.636930,0.750410,0.862250,1.223024,0.969826,0.727452,...,1.587078,2.134142,2.200722,1.389964,1.299770,1.242703,1.324041,1.558216,2.335192,2.169236
CA_3,1.554280,1.583142,1.241391,1.387996,1.251886,1.432929,1.542473,1.789439,1.830436,1.611020,...,1.982945,2.305346,2.613644,1.932109,1.811414,1.762873,1.830108,1.817645,2.319777,2.671040
CA_4,0.532962,0.582814,0.454575,0.472286,0.503772,0.455559,0.481797,0.652017,0.596261,0.503444,...,0.864218,1.067891,1.088226,0.991801,0.890456,0.826173,0.834372,0.886848,1.031814,1.179731
TX_1,0.838308,0.881273,0.597573,0.740571,0.555592,0.896687,0.554608,0.924893,0.946868,0.713021,...,1.183995,1.242047,1.639882,1.216136,1.085602,1.032142,0.901279,1.201705,1.366678,1.516563
TX_2,1.263365,1.291243,0.895704,0.968842,0.817317,1.127911,0.848803,1.237127,1.199410,0.961627,...,1.448016,1.450968,1.923909,1.370613,1.314529,1.153821,1.025254,1.393572,1.574943,1.711053
TX_3,0.993768,0.985897,0.729747,0.711381,0.566087,0.929157,0.638570,0.934077,0.928829,0.725812,...,1.408659,1.362742,1.765497,1.423745,1.328960,1.216792,1.078386,1.439816,1.545425,1.631355
WI_1,0.886848,0.719580,0.512299,0.410298,0.000656,0.672024,0.923254,1.065267,0.549032,0.444408,...,1.304690,1.812725,1.799934,1.149557,1.089866,1.063299,1.140702,1.250574,1.640538,1.657593
WI_2,0.739915,0.630371,0.661856,0.827156,0.385372,0.735979,0.732043,0.866842,0.701869,0.602165,...,1.743850,2.526730,2.488029,1.643490,1.517875,1.486717,1.517875,1.600525,1.709741,1.662184


In [76]:
sales.groupby(['store_id', 'dept_id']).mean()

d_1       d_2       d_3       d_4       d_5  \
store_id dept_id                                                         
CA_1     FOODS_1      1.375000  1.314815  0.990741  0.810185  0.842593   
         FOODS_2      1.693467  1.645729  0.994975  1.195980  0.889447   
         FOODS_3      2.755772  2.670717  1.698663  1.952612  1.817740   
         HOBBIES_1    1.269231  1.175481  0.983173  0.920673  0.632212   
         HOBBIES_2    0.187919  0.060403  0.040268  0.060403  0.033557   
...                        ...       ...       ...       ...       ...   
WI_3     FOODS_3      2.786148  2.895504  2.236938  2.387606  1.733900   
         HOBBIES_1    0.615385  0.822115  0.548077  0.439904  0.168269   
         HOBBIES_2    0.147651  0.093960  0.134228  0.073826  0.026846   
         HOUSEHOLD_1  1.097744  1.016917  0.789474  0.614662  0.283835   
         HOUSEHOLD_2  0.287379  0.378641  0.205825  0.182524  0.102913   

                           d_6       d_7       d_8       d_9      d_10  ...  \
store_id dept_id                                                        ...   
CA_1     FOODS_1      0.884259  1.037037  1.217593  1.134259  0.814815  ...   
         FOODS_2      1.221106  1.391960  1.876884  1.459799  1.404523  ...   
         FOODS_3      2.102066  2.326853  3.586877  3.149453  2.078979  ...   
         HOBBIES_1    1.088942  0.814904  1.802885  1.021635  0.901442  ...   
         HOBBIES_2    0.093960  0.046980  0.127517  0.046980  0.060403  ...   
...                        ...       ...       ...       ...       ...  ...   
WI_3     FOODS_3      3.353584  3.313487  4.600243  3.862697  2.337789  ...   
         HOBBIES_1    0.685096  0.858173  0.959135  0.519231  0.533654  ...   
         HOBBIES_2    0.053691  0.020134  0.087248  0.026846  0.140940  ...   
         HOUSEHOLD_1  0.736842  0.924812  1.216165  0.785714  0.665414  ...   
         HOUSEHOLD_2  0.236893  0.308738  0.361165  0.231068  0.209709  ...   

                        d_1932    d_1933    d_1934    d_1935    d_1936  \
store_id dept_id                                                         
CA_1     FOODS_1      1.907407  2.013889  1.921296  1.282407  1.398148   
         FOODS_2      1.369347  1.535176  1.962312  1.394472  1.195980   
         FOODS_3      2.925881  3.645200  4.278250  2.874848  2.449575   
         HOBBIES_1    1.312500  1.759615  1.516827  0.935096  0.834135   
         HOBBIES_2    0.281879  0.496644  0.375839  0.187919  0.308725   
...                        ...       ...       ...       ...       ...   
WI_3     FOODS_3      2.504253  3.420413  3.842041  2.371810  1.946537   
         HOBBIES_1    0.485577  0.829327  0.706731  0.449519  0.418269   
         HOBBIES_2    0.140940  0.248322  0.255034  0.073826  0.302013   
         HOUSEHOLD_1  1.142857  1.761278  1.642857  1.176692  1.122180   
         HOUSEHOLD_2  0.318447  0.398058  0.376699  0.267961  0.256311   

                        d_1937    d_1938    d_1939    d_1940    d_1941  
store_id dept_id                                                        
CA_1     FOODS_1      1.837963  1.527778  1.435185  1.976852  1.546296  
         FOODS_2      1.072864  1.032663  1.090452  1.575377  1.806533  
         FOODS_3      2.420413  2.567436  2.732685  3.315917  3.978129  
         HOBBIES_1    0.848558  0.850962  1.187500  1.576923  1.413462  
         HOBBIES_2    0.187919  0.348993  0.302013  0.422819  0.389262  
...                        ...       ...       ...       ...       ...  
WI_3     FOODS_3      1.936817  1.976914  2.443499  2.867558  3.198056  
         HOBBIES_1    0.689904  0.528846  0.581731  0.656250  0.733173  
         HOBBIES_2    0.302013  0.308725  0.181208  0.208054  0.248322  
         HOUSEHOLD_1  0.913534  1.118421  1.436090  1.359023  1.359023  
         HOUSEHOLD_2  0.258252  0.273786  0.446602  0.423301  0.403883  

[70 rows x 1941 columns]

In [77]:
level9 = sales.groupby(['store_id', 'dept_id']).mean()

In [78]:
calendar.to_csv('../M5_Dataset/new_calendar.csv')
prices.to_csv('../M5_Dataset/new_prices.csv')
sales.to_csv('../M5_Dataset/new_sales.csv')

level9.to_csv('../M5_Dataset/level9.csv')